In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls '/content/drive/My Drive/Final_Practicum/sentiment140'

All_Tweets_cleaned.csv		 Sentiment140_processing2.ipynb
All_Tweets_cleaned.ipynb	 Sentiment_Analysis_Dataset.csv
AllTweets.csv			 Sentiment_cleaned.csv
LSTM_Base			 Tfidf_Naive
model.h5			 Tweets.ipynb
senti_airline_naive_tfidf.ipynb


In [0]:
import re
import pandas as pd
import nltk
!pip install tweet-preprocessor
!pip install emoji --upgrade

from emoji import demojize
nltk.download('wordnet')
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.tokenize.stanford import StanfordTokenizer 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.5.0-cp36-none-any.whl size=7946 sha256=fc0c1e691e5bfb0d2a7077084c4e384835cc8b83d51fd8f478de59e8692b94e4
  Stored in directory: /root/.cache/pip/wheels/1b/27/cc/49938e98a2470802ebdefae9d2b3f524768e970c1ebbe2dc4a
Successfully built tweet-preprocessor
     |████████████████████████████████| 51kB 1.9MB/s 
  Created wheel for emoji: filename=emoji-0.5.3-cp36-none-any.whl size=42175 sha256=0762c76361866f94e3f2bb3ff41a69fed8c1e6373859086797764c50e61ec14a
  Stored in directory: /root/.cache/pip/wheels/86/09/26/f944015841423cd516e8a97f30e29be59e53461aea8b7d3458
Successfully built emoji
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
data = pd.read_csv('/content/drive/My Drive/Final_Practicum/sentiment140/AllTweets.csv', encoding="ISO-8859-1")
data.head()

,tweet_id,text
0,1.101630e+18,Modi has demonstrated that Kashmir is potentia...
1,1.101630e+18,Vande mataram sir.
2,1.101630e+18,You R Like Jhansi Ki Rani For All Nationalist ...
3,1.101630e+18,"If I run frm home still not leave me, making b..."
4,1.101630e+18,Meri oqqat hai ki mein ek Indian hu or teri oq...


In [0]:
data.shape

(64367, 2)

In [0]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
  
for index, row in data.iterrows():
  row['text']=deEmojify(str(row['text']))
  row['text'] = ' '.join(re.sub("(@[\w]*)|(#[A-Za-z0-9]+) | *RT*", " ", row['text']).split())
  row['text'] = ' '.join(re.sub("(\w+:\/\/\S+)", " ", row['text']).split())
  row['text'] = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", row['text']).split())
  row['text'] = row['text'].replace("[^a-zA-Z ]", "")

data['text']=data['text'].astype(str)
data['text'] = data['text'].str.strip()
data[data.isnull().any(axis=1)]
data.fillna('', inplace=True)
data['text'].head(10)



0    Modi has demonstrated that Kashmir is potentia...
1                                   Vande mataram sir.
2    You R Like Jhansi Ki Rani For All Nationalist ...
3    If I run frm home still not leave me, making b...
4    Meri oqqat hai ki mein ek Indian hu or teri oq...
5    Don't know how retaining #Abhi would have help...
6                     Jay hind  salute you  Abhinandan
7    @sherryontopp sir we have alot of respect for ...
8    @narendramodi @PMOIndia @RSSorg @BJP4India @Am...
9                   Vande besharam you are a joke modi
Name: text, dtype: object

In [0]:

def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    
    return text
# We are keeping cleaned tweets in a new column called 'tidy_tweets'
data['text'] = np.vectorize(remove_pattern)(data['text'], "@[\w]* | *RT*")
print(data['text'].head(10))

0    Modi has demonstrated that Kashmir is potentia...
1                                   Vande mataram sir.
2    You Like Jhansi Kiani For All Nationalist Indi...
3    If I run frm home still not leave me, making b...
4    Meri oqqat hai ki mein ek Indian hu or teri oq...
5    Don't know how retaining #Abhi would have help...
6                     Jay hind  salute you  Abhinandan
7    sir we have alot of respect for you if you had...
8    @UNICEFhttps://twitter.com/SampathT2/status/11...
9                   Vande besharam you are a joke modi
Name: text, dtype: object


In [0]:
data['text'] = data['text'].str.replace("[^a-zA-Z ]", "")
data.head(10)

,tweet_id,text
0,1.101630e+18,Modi has demonstrated that Kashmir is potentia...
1,1.101630e+18,Vande mataram sir
2,1.101630e+18,You Like Jhansi Kiani For All Nationalist Indi...
3,1.101630e+18,If I run frm home still not leave me making ba...
4,1.101630e+18,Meri oqqat hai ki mein ek Indian hu or teri oq...
5,1.101630e+18,Dont know how retaining Abhi would have helped...
6,1.101630e+18,Jay hind salute you Abhinandan
7,1.101630e+18,sir we have alot of respect for you if you had...
8,1.101630e+18,UNICEFhttpstwittercomSampathTstatus
9,1.101630e+18,Vande besharam you are a joke modi


In [0]:


for index, row in data.iterrows():
    # Here we are filtering out all the words that contains link
    text = [word for word in row.text.split() if 'http' not in word]
   
  data.head(10)

,tweet_id,text
0,1.101630e+18,Modi has demonstrated that Kashmir is potentia...
1,1.101630e+18,Vande mataram sir
2,1.101630e+18,You Like Jhansi Kiani For All Nationalist Indi...
3,1.101630e+18,If I run frm home still not leave me making ba...
4,1.101630e+18,Meri oqqat hai ki mein ek Indian hu or teri oq...
5,1.101630e+18,Dont know how retaining Abhi would have helped...
6,1.101630e+18,Jay hind salute you Abhinandan
7,1.101630e+18,sir we have alot of respect for you if you had...
8,1.101630e+18,UNICEFhttpstwittercomSampathTstatus
9,1.101630e+18,Vande besharam you are a joke modi


In [0]:
def in_dict(word):
    if wordnet.synsets(word):
        #if the word is in the dictionary, we'll return True
        return True

def replace_elongated_word(word):
    regex = r'(\w*)(\w+)\2(\w*)'
    repl = r'\1\2\3'    
    if in_dict(word):
        return word
    new_word = re.sub(regex, repl, word)
    if new_word != word:
        return replace_elongated_word(new_word)
    else:
        return new_word

def detect_elongated_words(row):
    regexrep = r'(\w*)(\w+)(\2)(\w*)'
    words = [''.join(i) for i in re.findall(regexrep, row)]
    for word in words:
        if not in_dict(word):
            row = re.sub(word, replace_elongated_word(word), row)
    return row

In [0]:
data['text'] = data['text'].apply(lambda x: detect_elongated_words(x))
data.head(10)

,tweet_id,text
0,1.101630e+18,Modi has demonstrated that Kashmir is potentia...
1,1.101630e+18,Vande mataram sir
2,1.101630e+18,You Like Jhansi Kiani For All Nationalist Indi...
3,1.101630e+18,If I run frm home still not leave me making ba...
4,1.101630e+18,Meri oqat hai ki mein ek Indian hu or teri oqa...
5,1.101630e+18,Dont know how retaining Abhi would have helped...
6,1.101630e+18,Jay hind salute you Abhinandan
7,1.101630e+18,sir we have alot of respect for you if you had...
8,1.101630e+18,UNICEFhtpstwitercomSampathTstatus
9,1.101630e+18,Vande besharam you are a joke modi


In [0]:
lemmatizer = WordNetLemmatizer() 
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [0]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:
cleaned_tweets=[]
for index,row in data.iterrows():
    stopWords = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(row["text"])
    rm_stopwords= [w for w in tokens if w not in stopWords]
    lemmatization=' '.join([lemmatizer.lemmatize(word , get_wordnet_pos(word)) for word in rm_stopwords if word])
    cleaned_tweets.append(lemmatization)
data['text']=pd.Series(cleaned_tweets)
data.head(10)

,tweet_id,text
0,1.101630e+18,Modi demonstrate Kashmir potentially flashpoin...
1,1.101630e+18,Vande mataram sir
2,1.101630e+18,You Like Jhansi Kiani For All Nationalist Indi...
3,1.101630e+18,If I run frm home still leave make battle life...
4,1.101630e+18,Meri oqat hai ki mein ek Indian hu teri oqat y...
5,1.101630e+18,Dont know retain Abhi would helpedeleasing put...
6,1.101630e+18,Jay hind salute Abhinandan
7,1.101630e+18,sir alot respect come pakistan say release ind...
8,1.101630e+18,UNICEFhtpstwitercomSampathTstatus
9,1.101630e+18,Vande besharam joke modi


In [0]:
data.to_csv('/content/drive/My Drive/Final_Practicum/sentiment140/All_Tweets_cleaned.csv', header=True, index=False)